# `System.Text.Json`: preserving escape characters


In [1]:
#!csharp

using System.Text.Json;

string json =
            """
            {
                "one": 1,
                "is-true": true,
                "three": [
                    "do not change this: \\d",
                    "do not change this either: |e",
                    "don't worry about this one"
                ]
            }
            """;

JsonElement root = JsonDocument.Parse(json.Replace(@"\", @"\\")).RootElement;

JsonElement[] jA = [.. root.GetProperty("three").EnumerateArray()];

jA

index,value
0,"""do not change this: \\d"""
1,"""do not change this either: |e"""
2,"""don't worry about this one"""


In [2]:
#!csharp

jA.Select(e => $@"""{e.GetString()}""").Aggregate((a,i) => $"{a},{Environment.NewLine}{i}")

"do not change this: \\d",
"do not change this either: |e",
"don't worry about this one"

## do not use the `JsonSerializer.Serialize` method 😐


The `JsonSerializer.Serialize` method produces text that preserves escape characters.

## C# by default escapes characters

In [3]:
#!csharp

string s = "do not escape this: \\\\d (and don't change this: | <-)";

s

do not escape this: \\d (and don't change this: | <-)

This default behavior can be a problem in the world of JSON:

In [4]:
#!csharp

using System.Text.Json;

string json =
    $$"""
    {
        "my-property": "{{s}}"
    }
    """;

JsonDocument.Parse(json)

my-property,"""do not escape this: \d (and don't change this: | <-)"""


To avoid this kind of `invalid escapable character within a JSON string` error, we use the `JsonSerializer.Serialize` method:

In [5]:
#!csharp

using System.Text.Json;

string json2 =
    $$"""
    {
        "my-property": {{JsonSerializer.Serialize(s)}}
    }
    """;

JsonDocument.Parse(json2)

my-property,"""do not escape this: \\d (and don't change this: | <-)"""


Notice there are no quotation marks after `"my-property":` as the JSON serializer will produce them:

In [6]:
#!csharp

JsonSerializer.Serialize(s)

"do not escape this: \\\\d (and don\u0027t change this: | \u003C-)"

## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼